In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def informacion(entry):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}

    url = f"https://www.uniprot.org/uniprot/?query={entry}+Bos+taurus&sort=score"
    r = requests.get(url, headers=headers)

    soup3 =BeautifulSoup(r.text, "html.parser")
    DUDA=soup3.find_all('div',{"id":"noResultsMessage"})
        
    if DUDA == []:
        soup =BeautifulSoup(r.text, "html.parser")
        questions =soup.find_all('tr', class_='entry selected-row')
        for item in questions:
            Entry = item.find('td',{'class':'entryID'}).text
            link='https://www.uniprot.org' +item.find('td', {'class': 'entryID'}).a['href']
            Entry_names = item.find('td',{'class':''}).text

        url2= f"https://www.uniprot.org/uniprot/{Entry}"
        r2 = requests.get(url2, headers=headers)
        soup2 =BeautifulSoup(r2.text, "html.parser")

        questions2 =soup2.find_all('div', {'class':'content entry_view_content up_entry swissprot'})

        for item in questions2:
            Protein = item.find('div',{'id':'content-protein'}).text
            Gene=item.find('div',{'id':'content-gene'}).text
            Organism=item.find('div',{'id':'content-organism'}).text
            Status=item.find('a',{'title':'Reviewed (Swiss-Prot)'}).text
        
        try:
            #print(Protein, Gene, Organism, Status) 
            datos1 = [Protein, Gene, Organism, Status]
        except UnboundLocalError:
            Protein = "Not found Protein"; Gene = "Not found Gene"
            Organism = "Not found Organism"; Status = "Not found Status" 
            #datos1 = [Protein, Gene, Organism, Status]
            
        go2=soup2.find("ul",{"class":"noNumbering molecular_function"})
        if go2 == None:
            print(" ")
        else :
            goes=go2.find_all("li")

        cadena2=""
        try:
            for j in goes:
                Molecular_function=str(j.find("a").text).capitalize()  #imprime todo texto de bio_process   
                cadena3=Molecular_function+"."
                cadena2=cadena2+cadena3
        except UnboundLocalError:
            cadena2 = "Cadena2 nor found "

        go=soup2.find("ul",{"class":"noNumbering biological_process"})
        if go == None:
            print(" ")
        else :
            goes=go.find_all("li")

        cadena=""
        try:
            for j in goes:
                Biological_process_row=str(j.find("a").text).capitalize()  #imprime todo texto de bio_process   
                cadena1=Biological_process_row+"."
                cadena=cadena+cadena1
        except UnboundLocalError:
            cadena = "Cadena1 not found"
        #KEYWORDS:"Molecular function', 'Biological process"
        linea1=[]; linea2=[]; linea3=[]
        division=soup2.find("div", {"id":"function"} )

        try:
            tabla=division.find("table", {"class":"databaseTable"})
            rows=tabla.find_all("tr")
        except AttributeError:
            molecular_f="Not found molecular_f"
            biological_p="Not found biological_p"
        else:
            for row in rows:
                linea1.append(str(row.find_all("td")[0].text))   #muestra ('Molecular function', 'Biological process')
                linea2.append(str(row.find_all("td")[1].text))   #muestra ("'Activator, DNA-binding','Transcription, Transcription regulation'")
                try:
                    linea3.append(str(row.find_all("td")[1].a.text))
                except AttributeError:
                    pass
            if len(linea1)==1:
                try:
                    linea3.append(str(row.find_all("td")[1].a.text))
                except AttributeError:
                    if len(linea3)!=0:
                        if linea1[0]=="Molecular function":
                            molecular_f = linea3[0]
                            biological_p="Not found biological_p"
                        if linea1[0]=="Biological process":
                            molecular_f="Not found biological_p"
                            biological_p= linea3[0]
                    else:
                        molecular_f="Not found molecular_f"
                        biological_p="Not found biological_p"
            if len(linea1)>=2:
                molecular_f=linea2[0]
                biological_p=linea2[1]
                #print(molecular_f,biological_p)
            if len(linea1)==0:
                molecular_f="Not found molecular_f"
                biological_p="Not found biological_p"
        
        try:
            datos = [Entry, Entry_names,Protein, Gene,Organism,Status,cadena2+cadena, molecular_f, biological_p]
        except UnboundLocalError:
            molecular_f="Not found molecular_f"
            biological_p="Not found biological_p"
            datos = [Entry, Entry_names,Protein, Gene,Organism,Status,cadena2+cadena, molecular_f, biological_p]
    else:
        datos = ["Gene not found"]*9
    # datos = [Entry, Entry_names,Protein, Gene,Organism,Status,cadena2+cadena, molecular_f, biological_p]
    return(datos)
#informacion("CD47")


In [ ]:

import xlrd
from xlutils.copy import copy

import xlwt

xl_file = "C:Reporte.xls"
rb = xlrd.open_workbook(xl_file, formatting_info=True)

style0 = xlwt.easyxf('font: name Verdana, colour black', "bold on")
wb = copy(rb) 

sheet = wb.get_sheet(0) 
book = xlrd.open_workbook("C:/Genesidentificadosconvecinos2.xlsx") 

sh = book.sheet_by_index(0) 

L = ["Gen Abrev", "Entry", "Entry Name", "Protein", "Gene", "Organism","Status", "GO - Biological process", "Molecular function", "Biological process"]
for k in range(len(L)):
    sheet.write(0,k, L[k], style0)
    
for i in range(1,10):
    m = informacion(sh.cell_value(rowx=i, colx=0))
    for j in range(len(m)): 
        sheet.write(i,0,sh.cell_value(rowx=i, colx=0)) 
        sheet.write(i,j+1,m[j])

wb.save(xl_file)